In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import time

position_dict = {'QB': 2, 'RB': 3, 'WR': 4, 'TE': 5, 'D/ST': 7, 'K': 8}
week_dict = {"NFL Week 1": 7,"NFL Week 2": 6, "NFL Week 3": 5, "NFL Week 4": 4}

driver_path = 'C:/Users/Daymo/GithubProjects/Selenium-Fantasy-Football/drivers/chromedriver-win64'  
driver = webdriver.Chrome()
# driver.implicitly_wait(1)
driver.get("https://fantasy.espn.com/football/leaders")
def extract_table_to_dataframe(header_xpath: str, tBody_xpath: str) -> pd.DataFrame:
    # Extract the column names from the header row
    header_row = driver.find_element(By.XPATH, header_xpath)
    cols = header_row.find_elements(By.TAG_NAME, 'th')
    column_names = [col.text for col in cols]
    df = pd.DataFrame(columns=column_names)
    tbody = driver.find_element(By.XPATH, tBody_xpath)
    rows = tbody.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        temp_list = [col.text for col in row.find_elements(By.TAG_NAME, 'td')]
        df.loc[len(df)] = temp_list
    return df

time.sleep(3)
label = driver.find_element(By.XPATH, f'//*[@id="filterSlotIds"]/label[{position_dict["K"]}]')
label.click()
time.sleep(3)
dropdown = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div/div[2]/select/option[{week_dict["NFL Week 1"]}]')
dropdown.click()
time.sleep(3)



In [22]:
final_df = pd.DataFrame(index=None)
for key, value in week_dict.items():
    dropdown = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div/div[2]/select/option[{value}]')
    dropdown.click()
    time.sleep(3)
    print(f'Start of {key}')
    while True: 
                i = 0
                try: 
                    time.sleep(5)
                    start = time.time()
                    header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/thead/tr[2]'
                    tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/tbody'
                    df1=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                    header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/thead/tr[2]'
                    tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/tbody'
                    df2=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                    header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/thead/tr[2]'
                    tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/tbody'
                    df3=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                    df_combined = pd.concat([df1, df2, df3], axis=1)
                    df_combined['week'] = str(key)

                    frames = [final_df, df_combined]
                    # final_df = pd(frames, ignore_index=True)
                    final_df = pd.concat(
                        frames,
                        axis=0,
                        join="outer",
                        ignore_index=True,

                    )

                    # button = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')
                    button = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')                                     
                    if button.is_enabled():
                        button.click()
                        print(f'Next page {time.time() - start}')
                    else:
                        print(f'End of Position {time.time() - start}')
                        break
                except Exception as e:
                    print(f'Excpetion {e}')
                    break

Start of NFL Week 1
Next page 4.324021339416504
End of Position 1.4690985679626465
Start of NFL Week 2
Next page 4.322043180465698
End of Position 1.5872910022735596
Start of NFL Week 3
Next page 4.327024698257446
End of Position 1.4875595569610596
Start of NFL Week 4
Next page 4.376901626586914
End of Position 1.4990324974060059


In [24]:
pd.set_option('display.max_rows', None)
final_df.head(268)

,PLAYER,TYPE,ACTION,OPP,STATUS,PROJ,FG39/FGA39,FG49/FGA49,FG50+/FGA50+,FG/FGA,XP/XPA,FPTS,week
0,Jake Elliott\nPhi\nK,,,@NE,W 25-20,8.3,1/1,1/1,2/2,4/4,1/2,18.0,NFL Week 1
1,Nick Folk\nTen\nK,,,@NO,L 15-16,7.5,3/3,1/1,1/1,5/5,0/0,18.0,NFL Week 1
2,Jake Moody\nSF\nK,,,@Pit,W 30-7,7.3,1/1,2/2,0/0,3/3,3/3,14.0,NFL Week 1
3,Jason Sanders\nMia\nK,,,@LAC,W 36-34,7.9,1/1,2/2,0/0,3/3,3/4,14.0,NFL Week 1
4,Tyler Bass\nBuf\nK,,,@NYJ,L 16-22,8.0,1/1,1/1,1/1,3/3,1/1,13.0,NFL Week 1
5,Brett Maher\nLAR\nK,,,@Sea,W 30-13,8.0,1/1,1/1,1/3,3/5,3/3,13.0,NFL Week 1
6,Cameron Dicker\nLAC\nK,,,Mia,L 34-36,8.0,1/1,0/0,1/1,2/2,4/4,12.0,NFL Week 1
7,Dustin Hopkins\nCle\nK,,,Cin,W 24-3,8.0,1/1,2/2,0/0,3/3,1/1,12.0,NFL Week 1
8,Matt Prater\nAri\nK,,,@Wsh,L 16-20,6.9,2/2,0/0,1/1,3/3,1/1,12.0,NFL Week 1
9,Blake Grupe\nNO\nK,,,Ten,W 16-15,7.5,2/2,0/0,1/1,3/3,1/1,12.0,NFL Week 1
